In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from sklearn.model_selection import train_test_split

from common import data
from common.training import training_loop

from algos import deep_fib

from common.models import resnet, deeplab, scinet

SCINET = "scinet"
DEEPLAB = "deeplab"

In [3]:
paths = data.get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [4]:
m_data_train = data.Marconi100Dataset(train, scaling=data.Scaling.MINMAX)
m_data_test = data.Marconi100Dataset(test, scaling=data.Scaling.MINMAX)


Loading: 100%|██████████| 25/25 [00:44<00:00,  1.78s/it]


In [5]:
model_name = DEEPLAB
horizon = 1024
stride = 512
n_masks = 50

batch_size = 32

log_dir = "./outputs/deep_fib_deeplab_sigmoid"
lr = 1e-3
num_epochs = 30

anomaly_threshold = 0.1  # to be tuned

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
dataset_train = data.UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)
dataset_test = data.UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

masks = deep_fib.get_masks(horizon, n_masks).float()

print(len(dataset_train), len(dataset_test), masks.size())


5644 647 torch.Size([50, 1024, 460])


In [7]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
print(len(train_loader), len(test_loader))


177 21


In [8]:
if model_name == SCINET:
    num_encoder_levels = 3
    hidden = [512]
    input_dim = 460
    hidden_size = 2
    kernel_size = 3
    dropout = 0.5

    model = scinet.SCINet(
        output_len=horizon,
        input_len=horizon,
        num_encoder_levels=num_encoder_levels,
        hidden_decoder_sizes=hidden,
        input_dim=input_dim,
        hidden_size=hidden_size,
        kernel_size=kernel_size,
        dropout=dropout,
    ).float()
    with open(f"{log_dir}/config.json", "w") as f:
        json.dump(
            dict(
                output_len=horizon,
                input_len=horizon,
                num_encoder_levels=num_encoder_levels,
                hidden_decoder_sizes=hidden,
                input_dim=input_dim,
                hidden_size=hidden_size,
                kernel_size=kernel_size,
                dropout=dropout,
            ),
            f,
        )
elif model_name == DEEPLAB:
    model = deeplab.DeepLabNet(
        resnet.ResNetFeatures(
            resnet.Bottleneck,
            resnet.RESNET50_LAYERS,
            return_layers=[resnet.LAYER_1, resnet.LAYER_4],
            replace_stride_with_dilation=[False, True, True],
            num_features=data.NUM_FEATURES,
        ),
        backbone_channels=[256, 2048],
        out_feats=data.NUM_FEATURES,
    ).float()
else:
    raise ValueError("Wrong model name")

print(model)

DeepLabNet(
  (backbone): ResNetFeatures(
    (conv1): Conv1d(460, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layers): ModuleList(
      (0): Sequential(
        (0): Bottleneck(
          (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
          (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=reflect)
          (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
          (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv1d(64, 256, kernel_siz

In [9]:
import gc

gc.collect()

0

In [10]:
engine = deep_fib.DeepFIBEngine(anomaly_threshold, masks)
optim = Adam(model.parameters(), lr=lr)
lr_sched = CosineAnnealingLR(optim, num_epochs)

with SummaryWriter(log_dir) as writer:
    training_loop(
        model=model,
        engine=engine,
        num_epochs=num_epochs,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        device=device,
        optimizer=optim,
        lr_scheduler=lr_sched,
        writer=writer,
        save_path=log_dir + "/models",
    )

Epoch 0 - train_loss = 0.055 - test_loss = 0.056 - lr = 1.00e-03


Epoch 1 - train_loss = 0.050 - test_loss = 0.049 - lr = 9.97e-04


Epoch 2 - train_loss = 0.047 - test_loss = 0.048 - lr = 9.89e-04


Epoch 3 - train_loss = 0.044 - test_loss = 0.043 - lr = 9.76e-04


Epoch 4 - train_loss = 0.042 - test_loss = 0.044 - lr = 9.57e-04


...

Epoch 25 - train_loss = 0.033 - test_loss = 0.036 - lr = 6.70e-05


Epoch 26 - train_loss = 0.032 - test_loss = 0.035 - lr = 4.32e-05


Epoch 27 - train_loss = 0.033 - test_loss = 0.035 - lr = 2.45e-05


Epoch 28 - train_loss = 0.032 - test_loss = 0.036 - lr = 1.09e-05


Epoch 29 - train_loss = 0.032 - test_loss = 0.035 - lr = 2.74e-06
